# Engenharia de Atributos

In [37]:
import pandas as pd

In [38]:
merged_dataset = pd.read_csv('../data/processed/merged_dataset.csv')

/tmp/ipykernel_59660/3999875462.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_dataset = pd.read_csv('../data/processed/merged_dataset.csv')


Convertendo as variáveis para os seus respectivos tipos corretos 

In [39]:
# Conversão dos tipos numéricos e categóricos
merged_dataset = merged_dataset.astype({
    'ID_CLIENTE': 'int64',
    'VALOR_A_PAGAR': 'float64',
    'TAXA': 'float64',
    'RENDA_MES_ANTERIOR': 'float64',
    'NO_FUNCIONARIOS': 'int64',
    'FLAG_PF': 'int', 
    'DIAS_ATRASO': 'Int64',
    'TARGET_INADIMPLENCIA': 'int',
    'SEGMENTO_INDUSTRIAL': 'category',
    'PORTE': 'category',
    'CEP_2_DIG': 'category',
    'DIAS_ADIANTAMENTO': 'int',
    'DOMINIO_EMAIL': 'category',
})

# Conversão de datas
merged_dataset['SAFRA_REF'] = pd.to_datetime(
    merged_dataset['SAFRA_REF'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(
    merged_dataset['DATA_EMISSAO_DOCUMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_PAGAMENTO'] = pd.to_datetime(
    merged_dataset['DATA_PAGAMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_VENCIMENTO'] = pd.to_datetime(
    merged_dataset['DATA_VENCIMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_CADASTRO'] = pd.to_datetime(
    merged_dataset['DATA_CADASTRO'], format='%Y-%m-%d', errors='coerce'
)


### Criando novas features

Calculando tempo de casa. Indica a relação dos clientes com a empresa, isso pode influenciar na inadimplência. Clientes mais velhos de casa têm menos tendência a inadimplência.

In [40]:
merged_dataset['TEMPO_CADASTRO_PARA_VENCIMENTO'] = (merged_dataset['DATA_VENCIMENTO'] - merged_dataset['DATA_CADASTRO']).dt.days

In [41]:
merged_dataset['TEMPO_DE_CASA_MESES'] = merged_dataset['TEMPO_CADASTRO_PARA_VENCIMENTO'] // 30


In [42]:
merged_dataset = merged_dataset.drop(columns='TEMPO_CADASTRO_PARA_VENCIMENTO')


Prazo para o pagamento: ajuda a entender o tempo que o cliente teve para se preparar para o pagamento. Prazos curtos podem influenciar na inadimplência.



In [43]:
merged_dataset['PRAZO_PAGAMENTO_DIAS'] = (merged_dataset['DATA_VENCIMENTO'] - merged_dataset['DATA_EMISSAO_DOCUMENTO']).dt.days

Mês da SAFRA_REF: capta padrões sazonais (ex: pode haver mais inadimplência no final do ano ou em meses com impostos).

In [44]:
merged_dataset['MES_SAFRA'] = merged_dataset['SAFRA_REF'].dt.month

Criação de **INADIMPLENCIAS_ANTERIORES**. Essa variável é um contador cumulativo do número de inadimplências passadas para cada cliente até o momento do pagamento atual. Ela captura o comportamento passado do cliente, que é um forte indicador de risco de inadimplência. Clientes reincidentes tendem a ter maior probabilidade de inadimplir novamente.

In [ ]:
# PODE CONSERTAR ESSE CÓDIGO QUE ALGO DE ERRADO NÃO ESTÁ CERTO...

# Ordena por cliente e data de emissão
merged_dataset = merged_dataset.sort_values(by=['ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO'])

# Cria coluna cumulativa de inadimplências
merged_dataset['INADIMPLENCIAS_ANTERIORES'] = (
    merged_dataset
    .groupby('ID_CLIENTE')['TARGET_INADIMPLENCIA']
    .cumsum()
    .shift(fill_value=0)  # para não contar a linha atual, evitando data leakage
)


In [46]:
merged_dataset.isnull().sum()

ID_CLIENTE                   0
SAFRA_REF                    0
DATA_EMISSAO_DOCUMENTO       0
DATA_PAGAMENTO               0
DATA_VENCIMENTO              0
VALOR_A_PAGAR                0
TAXA                         0
RENDA_MES_ANTERIOR           0
NO_FUNCIONARIOS              0
DATA_CADASTRO                0
FLAG_PF                      0
SEGMENTO_INDUSTRIAL          0
DOMINIO_EMAIL                0
PORTE                        0
CEP_2_DIG                    0
DIAS_ATRASO                  0
TARGET_INADIMPLENCIA         0
DIAS_ADIANTAMENTO            0
TEMPO_DE_CASA_MESES          0
PRAZO_PAGAMENTO_DIAS         0
MES_SAFRA                    0
INADIMPLENCIAS_ANTERIORES    0
dtype: int64

Criação da coluna ADIANTAMENTOS_ANTERIORES. Usar o histórico de adiantamentos como feature pode ser útil em modelos de inadimplência, pois clientes que pagam antes do vencimento tendem a apresentar menor risco, revelando disciplina financeira e previsibilidade de caixa. Para evitar **data leakage**, a contagem considera apenas os pagamentos realizados antes da data da cobrança analisada, sem utilizar informações futuras que estariam indisponíveis no momento da previsão.


In [ ]:
# PODE CONSERTAR ESSE CÓDIGO QUE ALGO DE ERRADO NÃO ESTÁ CERTO...

merged_dataset['ADIANTAMENTOS_ANTERIORES'] = (
    merged_dataset
    .groupby('ID_CLIENTE')['TARGET_INADIMPLENCIA']
    .cumsum()
    .shift(fill_value=0)  # para não contar a linha atual, evitando data leakage
)


Selecionando uma amostra para conferir se o que eu fiz funcionou corretamente 

In [48]:
merged_dataset[merged_dataset['ID_CLIENTE']==8784237149961904][['DATA_EMISSAO_DOCUMENTO','DATA_VENCIMENTO','DATA_PAGAMENTO','DIAS_ADIANTAMENTO', 'ADIANTAMENTOS_ANTERIORES', 'INADIMPLENCIAS_ANTERIORES', 'TARGET_INADIMPLENCIA']]

,DATA_EMISSAO_DOCUMENTO,DATA_VENCIMENTO,DATA_PAGAMENTO,DIAS_ADIANTAMENTO,ADIANTAMENTOS_ANTERIORES,INADIMPLENCIAS_ANTERIORES,TARGET_INADIMPLENCIA
0,2018-09-04,2018-09-24,2018-09-24,0,0,0,0
1,2018-09-06,2018-09-24,2018-09-24,0,0,0,0
2,2018-09-09,2018-09-25,2018-09-25,0,0,0,0
3,2018-09-11,2018-09-27,2018-09-27,0,0,0,0
4,2018-09-17,2018-10-03,2018-10-03,0,0,0,0
...,...,...,...,...,...,...,...
231,2021-06-10,2021-06-28,2021-06-28,0,1,1,0
232,2021-06-11,2021-07-01,2021-07-01,0,1,1,0
233,2021-06-12,2021-06-29,2021-06-29,0,1,1,0
234,2021-06-14,2021-06-30,2021-06-30,0,1,1,0


Analisando a correlação de todas as features com a variável target

In [ ]:
#merged_dataset.to_csv('../data/processed/dataset_features_v1.csv', index=False)

# LIDAR COM A ALTA CARDINALIDADE DE CEP_2_DIG POSTERIORMENTE
